# Functional Autoencoder(2025Fall Seminar)

### Functional Autoencoder realization

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from scipy.interpolate import BSpline, splrep
import matplotlib.pyplot as plt


class FunctionalAutoencoder(nn.Module):
    def __init__(self, t_grid, num_basis=20, hidden_dim=32, hidden_dim2=16, latent_dim=8, degree=3):
        super(FunctionalAutoencoder, self).__init__()
        self.t_grid = t_grid         
        self.M = len(t_grid)
        self.dt = t_grid[1] - t_grid[0]  # assume uniform grid
        self.num_basis = num_basis
        self.degree = degree
        self.latent_dim = latent_dim
        self.hidden_dim = hidden_dim
        self.hidden_dim2 = hidden_dim2

        # B-spline basis (uniform, clamped at the endpoints)
        self.knots = self._make_clamped_uniform_knots(num_basis=self.num_basis, degree=self.degree)
        basis_matrix = self._compute_basis_matrix()  # shape (num_basis, M)
        self.register_buffer('basis_matrix', basis_matrix)

        # Encoder functional weights: each hidden unit has num_basis coefficients
        # w_k(t) = sum_l c_kl * B_l(t)
        self.encoder_coeffs = nn.Parameter(torch.randn(hidden_dim, num_basis) * 0.01)

        # Encoder fully-connected layers: integrals -> hidden_dim -> hidden_dim2 -> latent_dim
        self.fc_encoder_1 = nn.Linear(hidden_dim, hidden_dim2)
        self.fc_encoder_2 = nn.Linear(hidden_dim2, latent_dim)

        # Decoder fully-connected layers: latent_dim -> hidden_dim2 -> hidden_dim
        self.fc_decoder_1 = nn.Linear(latent_dim, hidden_dim2)
        self.fc_decoder_2 = nn.Linear(hidden_dim2, hidden_dim)

        # Decoder functional weights: each hidden unit has num_basis coefficients
        # v_k(t) = sum_l d_kl * B_l(t)
        self.decoder_coeffs = nn.Parameter(torch.randn(hidden_dim, num_basis) * 0.01)

        self.activation = nn.ReLU()

    def _make_clamped_uniform_knots(self, num_basis, degree):
        n = num_basis
        k = degree
        if n <= k:
            raise ValueError("num_basis must be greater than degree")
        num_internal = n - k - 1
        if num_internal > 0:
            internal = np.linspace(0.0, 1.0, num_internal + 2)[1:-1]
        else:
            internal = np.array([])
        t = np.concatenate([
            np.zeros(k + 1),
            internal,
            np.ones(k + 1)
        ])
        return t

    def _compute_basis_matrix(self):
        """Evaluate B-spline basis functions on the grid, shape (num_basis, M)."""
        basis_matrix = np.zeros((self.num_basis, self.M))
        eye = np.eye(self.num_basis)
        for l in range(self.num_basis):
            spline = BSpline(self.knots, eye[l], k=self.degree, extrapolate=True)
            basis_matrix[l] = spline(self.t_grid)
        return torch.tensor(basis_matrix, dtype=torch.float32)

    def forward(self, x):
        """x: input functional data, shape (batch_size, M)."""
        # Encoder: functional inner products -> FC layers -> latent
        w_grid = self.encoder_coeffs @ self.basis_matrix        # (hidden_dim, M)
        integrals = torch.einsum('bm,km->bk', x, w_grid) * self.dt

        hidden1 = self.activation(integrals)
        hidden2 = self.activation(self.fc_encoder_1(hidden1))
        latent = self.fc_encoder_2(hidden2)

        # Decoder: latent -> FC layers -> functional reconstruction
        hidden2_dec = self.activation(self.fc_decoder_1(latent))
        hidden1_dec = self.activation(self.fc_decoder_2(hidden2_dec))

        v_grid = self.decoder_coeffs @ self.basis_matrix        # (hidden_dim, M)
        recon = torch.einsum('bk,km->bm', hidden1_dec, v_grid)

        return recon, latent


def train_fae(
    data,
    t_grid,
    epochs=500,
    lr=0.005,
    batch_size=32,
    num_basis=20,
    hidden_dim=32,
    hidden_dim2=16,
    latent_dim=8,
    degree=3
):
    """
    Train the functional autoencoder on discretized functional data.

    data: numpy array of shape (n_samples, M)
    t_grid: numpy array of shape (M,)
    """
    model = FunctionalAutoencoder(
        t_grid,
        num_basis=num_basis,
        hidden_dim=hidden_dim,
        hidden_dim2=hidden_dim2,
        latent_dim=latent_dim,
        degree=degree
    )
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss(reduction='sum')

    dataset = torch.utils.data.TensorDataset(torch.tensor(data, dtype=torch.float32))
    loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

    print("\n--- Training FAE ---")
    final_loss = 0.0

    for epoch in range(epochs):
        model.train()
        total_loss = 0.0
        for batch in loader:
            x = batch[0] 
            recon, _ = model(x)
            loss = criterion(recon, x)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_loss = total_loss / len(loader.dataset)
        if (epoch + 1) % 10 == 0 or epoch == epochs - 1:
            print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}")
        if epoch == epochs - 1:
            final_loss = avg_loss

    return model, final_loss


#### FPCA realization

In [2]:
import numpy as np
from skfda.representation.grid import FDataGrid
from skfda.preprocessing.dim_reduction import FPCA

def train_fpca_skfda(data, t_grid, n_components):
    fd = FDataGrid(data_matrix=data, grid_points=t_grid)
    fpca = FPCA(n_components=n_components)
    fpca.fit(fd)
    scores = fpca.transform(fd)
    fd_recon = fpca.inverse_transform(scores)
    recon = fd_recon.data_matrix[:, 0, :]
    mse = np.mean((recon - data) ** 2)
    return fpca, scores, recon, mse



#### Data Generation

In [3]:
import numpy as np

def simulate_case(case: int, n: int = 100, T: int = 51, delta: float = 0.1, seed: int = None):
    if seed is not None:
        np.random.seed(seed)

    # Observation grid: 51 equally spaced points on [0,1]
    t_grid = np.linspace(0.0, 1.0, T)

    X = np.zeros((n, T))
    Y = np.zeros((n, T))

    for i in range(n):
        # Generate ξ_{i1}, ξ_{i2} based on different cases
        if case == 1:
            # ξ_{i1} ~ N(0, 3^2), ξ_{i2} ~ N(0, 2^2)
            xi1 = np.random.normal(0.0, 3.0)
            xi2 = np.random.normal(0.0, 2.0)
            Xi = xi1 * np.sin(2 * np.pi * t_grid) + xi2 * np.cos(2 * np.pi * t_grid)

        elif case == 2:
            # ξ_{i1}, ξ_{i2} ~ N(0, 2^2)
            xi1 = np.random.normal(0.0, 2.0)
            xi2 = np.random.normal(0.0, 2.0)
            Xi = xi2 * np.sin(xi1 * t_grid)

        elif case == 3:
            # Same distribution as Case 2
            xi1 = np.random.normal(0.0, 2.0)
            xi2 = np.random.normal(0.0, 2.0)
            Xi = xi2 * np.cos(xi1 * t_grid)

        elif case == 4:
            # ξ_{i1}, ξ_{i2} ~ N(0, 2^2)
            xi1 = np.random.normal(0.0, 2.0)
            xi2 = np.random.normal(0.0, 2.0)
            Xi = (
                xi1 * np.sin(2 * np.pi * t_grid)
                + xi2 * np.cos(2 * np.pi * t_grid)
                + xi2 * np.sin(xi1 * t_grid)
            )

        elif case == 5:
            # ξ_{i1}, ξ_{i2} ~ N(0, 2^2)
            xi1 = np.random.normal(0.0, 2.0)
            xi2 = np.random.normal(0.0, 2.0)
            Xi = (
                xi1 * np.sin(2 * np.pi * t_grid)
                + xi2 * np.cos(2 * np.pi * t_grid)
                + xi2 * np.cos(xi1 * t_grid)
            )

        # Save true curve
        X[i, :] = Xi

        # Add measurement error ε_ij ~ N(0, δ^2)
        eps = np.random.normal(0.0, delta, size=T)
        Y[i, :] = Xi + eps

    return t_grid, X, Y



#### Reconstruction Comparison

In [4]:
import pandas as pd
import torch

# Hyperparameters
n_samples = 100
T = 51
delta = 0.1
n_runs = 5
cases = [1, 2, 3, 4, 5]

# FAE parameters
fae_params = {
    'epochs': 300,
    'lr': 0.005,
    'batch_size': 32,
    'num_basis': 20,
    'hidden_dim': 32,
    'hidden_dim2': 16,
    'latent_dim': 8,
    'degree': 3
}

# FPCA parameters
fpca_n_components = 8 

# Store results
results = {
    'Case': [],
    'FAE_Loss_Mean': [],
    'FAE_Loss_Std': [],
    'FPCA_Loss_Mean': [],
    'FPCA_Loss_Std': []
}

print("=" * 60)
print("Comparing FAE and FPCA Reconstruction Loss")
print("=" * 60)

for case in cases:
    print(f"\n{'='*60}")
    print(f"Case {case}")
    print(f"{'='*60}")
    
    fae_losses = []
    fpca_losses = []
    
    for run in range(n_runs):
        print(f"\n--- Run {run + 1}/{n_runs} ---")
        
        seed = case * 100 + run
        t_grid, X_true, Y_obs = simulate_case(case=case, n=n_samples, T=T, delta=delta, seed=seed)
        
        # Train FAE
        print("Training FAE...")
        fae_model, fae_loss = train_fae(
            data=Y_obs,
            t_grid=t_grid,
            **fae_params
        )
        fae_losses.append(fae_loss)
        print(f"FAE Loss: {fae_loss:.6f}")
        
        # Train FPCA
        print("Training FPCA...")
        fpca_model, fpca_scores, fpca_recon, fpca_loss = train_fpca_skfda(
            data=Y_obs,
            t_grid=t_grid,
            n_components=fpca_n_components
        )
        fpca_losses.append(fpca_loss)
        print(f"FPCA Loss: {fpca_loss:.6f}")
    
    # Calculate statistics
    fae_mean = np.mean(fae_losses)
    fae_std = np.std(fae_losses)
    fpca_mean = np.mean(fpca_losses)
    fpca_std = np.std(fpca_losses)
    
    results['Case'].append(case)
    results['FAE_Loss_Mean'].append(fae_mean)
    results['FAE_Loss_Std'].append(fae_std)
    results['FPCA_Loss_Mean'].append(fpca_mean)
    results['FPCA_Loss_Std'].append(fpca_std)
    
    print(f"\n{'='*60}")
    print(f"Case {case} Summary:")
    print(f"FAE:  {fae_mean:.6f} ± {fae_std:.6f}")
    print(f"FPCA: {fpca_mean:.6f} ± {fpca_std:.6f}")
    print(f"{'='*60}")

# Create and display results table
print("\n\n" + "=" * 80)
print("FINAL RESULTS: Reconstruction Loss Comparison (MSE)")
print("=" * 80)

df = pd.DataFrame(results)
df['FAE_Loss'] = df['FAE_Loss_Mean'].apply(lambda x: f"{x:.6f}") + " ± " + df['FAE_Loss_Std'].apply(lambda x: f"{x:.6f}")
df['FPCA_Loss'] = df['FPCA_Loss_Mean'].apply(lambda x: f"{x:.6f}") + " ± " + df['FPCA_Loss_Std'].apply(lambda x: f"{x:.6f}")
df['Winner'] = df.apply(lambda row: 'FAE' if row['FAE_Loss_Mean'] < row['FPCA_Loss_Mean'] else 'FPCA', axis=1)

# Display the formatted table
result_table = df[['Case', 'FAE_Loss', 'FPCA_Loss', 'Winner']]
print(result_table.to_string(index=False))
print("=" * 80)

# Additional statistics
fae_wins = (df['Winner'] == 'FAE').sum()
fpca_wins = (df['Winner'] == 'FPCA').sum()
print(f"\nSummary: FAE wins in {fae_wins} cases, FPCA wins in {fpca_wins} cases")
print("=" * 80)


Comparing FAE and FPCA Reconstruction Loss

Case 1

--- Run 1/5 ---
Training FAE...

--- Training FAE ---
Epoch 10, Loss: 225.0836
Epoch 20, Loss: 162.3332
Epoch 30, Loss: 96.2664
Epoch 40, Loss: 25.9556
Epoch 50, Loss: 2.6854
Epoch 60, Loss: 8.5345
Epoch 70, Loss: 0.9257
Epoch 80, Loss: 0.7692
Epoch 90, Loss: 0.7001
Epoch 100, Loss: 0.6806
Epoch 110, Loss: 1.3357
Epoch 120, Loss: 0.7680
Epoch 130, Loss: 1.1014
Epoch 140, Loss: 0.6171
Epoch 150, Loss: 0.5930
Epoch 160, Loss: 0.9308
Epoch 170, Loss: 2.3322
Epoch 180, Loss: 0.7030
Epoch 190, Loss: 0.6343
Epoch 200, Loss: 0.5561
Epoch 210, Loss: 0.6183
Epoch 220, Loss: 3.9192
Epoch 230, Loss: 0.6008
Epoch 240, Loss: 0.6575
Epoch 250, Loss: 1.1893
Epoch 260, Loss: 0.6348
Epoch 270, Loss: 0.5211
Epoch 280, Loss: 0.5200
Epoch 290, Loss: 0.7235
Epoch 300, Loss: 2.1958
FAE Loss: 2.195778
Training FPCA...
FPCA Loss: 10.327919

--- Run 2/5 ---
Training FAE...

--- Training FAE ---
Epoch 10, Loss: 295.9275
Epoch 20, Loss: 29.2329
Epoch 30, Loss: 

#### Data Loader from UCR

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from scipy.optimize import linear_sum_assignment

def clustering_accuracy(y_true, y_pred):
    y_true = y_true.astype(np.int64)
    y_pred = y_pred.astype(np.int64)
    assert y_pred.size == y_true.size
    D = max(y_pred.max(), y_true.max()) + 1
    w = np.zeros((D, D), dtype=np.int64)
    for i in range(y_pred.size):
        w[y_pred[i], y_true[i]] += 1
    
    row_ind, col_ind = linear_sum_assignment(w.max() - w)
    accuracy = w[row_ind, col_ind].sum() / y_pred.size
    return accuracy


def evaluate_clustering(embeddings, true_labels, n_clusters, n_runs=10):

    acc_list = []
    ari_list = []
    
    for _ in range(n_runs):
        kmeans = KMeans(n_clusters=n_clusters, n_init=10, random_state=None)
        pred_labels = kmeans.fit_predict(embeddings)
        
        acc = clustering_accuracy(true_labels, pred_labels)
        ari = adjusted_rand_score(true_labels, pred_labels)
        
        acc_list.append(acc)
        ari_list.append(ari)
    
    return max(acc_list), max(ari_list), np.mean(acc_list), np.mean(ari_list)

print("Clustering evaluation functions loaded.")


Clustering evaluation functions loaded.


In [ ]:
from aeon.datasets import load_classification
from sklearn.preprocessing import StandardScaler

def load_ucr_dataset(dataset_name):

    print(f"Loading {dataset_name}...")
    
    # Load train and test data
    X_train, y_train = load_classification(dataset_name, split="train")
    X_test, y_test = load_classification(dataset_name, split="test")
    
    # Convert to numpy arrays if needed
    if hasattr(X_train, 'to_numpy'):
        X_train = X_train.to_numpy()
    if hasattr(X_test, 'to_numpy'):
        X_test = X_test.to_numpy()
    
    # Squeeze if multivariate (we'll use first dimension for univariate)
    if len(X_train.shape) == 3:
        X_train = X_train[:, 0, :]  # Take first dimension
        X_test = X_test[:, 0, :]
    
    # Merge train and test
    data = np.vstack([X_train, X_test])
    labels = np.concatenate([y_train, y_test])
    
    # Create time grid
    n_timepoints = data.shape[1]
    t_grid = np.linspace(0.0, 1.0, n_timepoints)
    
    print(f"  Dataset shape: {data.shape}")
    print(f"  Number of samples: {len(data)}")
    print(f"  Number of time points: {n_timepoints}")
    print(f"  Number of classes: {len(np.unique(labels))}")
    
    return data, labels, t_grid


# Load UCR datasets
ucr_datasets = {
    'BME': 'BME',
    'DiatomSizeReduction': 'DiatomSizeReduction',
    'Plane': 'Plane', 
    'Fungi': 'Fungi',
}

# Store all datasets
datasets_dict = {}
for short_name, full_name in ucr_datasets.items():
    try:
        data, labels, t_grid = load_ucr_dataset(full_name)
        datasets_dict[short_name] = {
            'data': data,
            'labels': labels,
            't_grid': t_grid
        }
    except Exception as e:
        print(f"Error loading {full_name}: {e}")
        print(f"Skipping {short_name}...")

print(f"\nSuccessfully loaded {len(datasets_dict)} datasets")


Loading BME...
  Dataset shape: (180, 128)
  Number of samples: 180
  Number of time points: 128
  Number of classes: 3
Loading DiatomSizeReduction...
  Dataset shape: (322, 345)
  Number of samples: 322
  Number of time points: 345
  Number of classes: 4
Loading Plane...
  Dataset shape: (210, 144)
  Number of samples: 210
  Number of time points: 144
  Number of classes: 7
Loading Fungi...
  Dataset shape: (204, 201)
  Number of samples: 204
  Number of time points: 201
  Number of classes: 18

Successfully loaded 4 datasets


### Comparison of cluster results

In [ ]:
import pandas as pd
import torch

# FAE parameters for UCR datasets
fae_params = {
    'epochs': 500,
    'lr': 0.005,
    'batch_size': 32,
    'num_basis': 20,
    'hidden_dim': 32,
    'hidden_dim2': 16,
    'latent_dim': 8,
    'degree': 3
}

# FPCA parameters
fpca_n_components = 8
n_model_runs = 5  
n_kmeans_runs = 10  

# Store results
results = {
    'Dataset': [],
    'N_Samples': [],
    'N_Classes': [],
    'N_Timepoints': [],
    'FAE_ACC_Mean': [],
    'FAE_ACC_Std': [],
    'FAE_ARI_Mean': [],
    'FAE_ARI_Std': [],
    'FPCA_ACC_Mean': [],
    'FPCA_ACC_Std': [],
    'FPCA_ARI_Mean': [],
    'FPCA_ARI_Std': []
}

print("=" * 100)
print("Comparing FAE and FPCA Clustering Performance on UCR Datasets")
print("=" * 100)

for dataset_name in datasets_dict.keys():
    print(f"\n{'='*100}")
    print(f"Dataset: {dataset_name}")
    print(f"{'='*100}")
    
    # Get dataset
    data = datasets_dict[dataset_name]['data']
    labels = datasets_dict[dataset_name]['labels']
    t_grid = datasets_dict[dataset_name]['t_grid']
    
    # Convert labels to integers starting from 0
    unique_labels = np.unique(labels)
    label_mapping = {old_label: new_label for new_label, old_label in enumerate(unique_labels)}
    labels_int = np.array([label_mapping[label] for label in labels])
    n_clusters = len(unique_labels)
    
    print(f"Shape: {data.shape}")
    print(f"Classes: {n_clusters}")
    
    fae_acc_list = []
    fae_ari_list = []
    fpca_acc_list = []
    fpca_ari_list = []
    
    for run in range(n_model_runs):
        print(f"\n--- Model Training Run {run + 1}/{n_model_runs} ---")
        
        # Train FAE and extract latent representations
        print("Training FAE...")
        fae_model, fae_loss = train_fae(
            data=data,
            t_grid=t_grid,
            **fae_params
        )
        
        # Get FAE latent representations
        fae_model.eval()
        with torch.no_grad():
            data_tensor = torch.tensor(data, dtype=torch.float32)
            _, fae_latent = fae_model(data_tensor)
            fae_latent_np = fae_latent.cpu().numpy()
        
        # Evaluate FAE clustering
        fae_best_acc, fae_best_ari, fae_mean_acc, fae_mean_ari = evaluate_clustering(
            fae_latent_np, labels_int, n_clusters, n_runs=n_kmeans_runs
        )
        fae_acc_list.append(fae_best_acc)
        fae_ari_list.append(fae_best_ari)
        print(f"FAE - ACC: {fae_best_acc:.4f}, ARI: {fae_best_ari:.4f}")
        
        # Train FPCA and extract scores
        print("Training FPCA...")
        fpca_model, fpca_scores, fpca_recon, fpca_loss = train_fpca_skfda(
            data=data,
            t_grid=t_grid,
            n_components=fpca_n_components
        )
        
        # Evaluate FPCA clustering
        fpca_best_acc, fpca_best_ari, fpca_mean_acc, fpca_mean_ari = evaluate_clustering(
            fpca_scores, labels_int, n_clusters, n_runs=n_kmeans_runs
        )
        fpca_acc_list.append(fpca_best_acc)
        fpca_ari_list.append(fpca_best_ari)
        print(f"FPCA - ACC: {fpca_best_acc:.4f}, ARI: {fpca_best_ari:.4f}")
    
    # Calculate statistics
    fae_acc_mean = np.mean(fae_acc_list)
    fae_acc_std = np.std(fae_acc_list)
    fae_ari_mean = np.mean(fae_ari_list)
    fae_ari_std = np.std(fae_ari_list)
    
    fpca_acc_mean = np.mean(fpca_acc_list)
    fpca_acc_std = np.std(fpca_acc_list)
    fpca_ari_mean = np.mean(fpca_ari_list)
    fpca_ari_std = np.std(fpca_ari_list)
    
    results['Dataset'].append(dataset_name)
    results['N_Samples'].append(data.shape[0])
    results['N_Classes'].append(n_clusters)
    results['N_Timepoints'].append(data.shape[1])
    results['FAE_ACC_Mean'].append(fae_acc_mean)
    results['FAE_ACC_Std'].append(fae_acc_std)
    results['FAE_ARI_Mean'].append(fae_ari_mean)
    results['FAE_ARI_Std'].append(fae_ari_std)
    results['FPCA_ACC_Mean'].append(fpca_acc_mean)
    results['FPCA_ACC_Std'].append(fpca_acc_std)
    results['FPCA_ARI_Mean'].append(fpca_ari_mean)
    results['FPCA_ARI_Std'].append(fpca_ari_std)
    
    print(f"\n{'='*100}")
    print(f"{dataset_name} Summary:")
    print(f"FAE  - ACC: {fae_acc_mean:.4f} ± {fae_acc_std:.4f}, ARI: {fae_ari_mean:.4f} ± {fae_ari_std:.4f}")
    print(f"FPCA - ACC: {fpca_acc_mean:.4f} ± {fpca_acc_std:.4f}, ARI: {fpca_ari_mean:.4f} ± {fpca_ari_std:.4f}")
    print(f"Winner (ACC): {'FAE' if fae_acc_mean > fpca_acc_mean else 'FPCA'}")
    print(f"Winner (ARI): {'FAE' if fae_ari_mean > fpca_ari_mean else 'FPCA'}")
    print(f"{'='*100}")

# Create and display results table
print("\n\n" + "=" * 120)
print("FINAL RESULTS: UCR Datasets Clustering Performance Comparison")
print("=" * 120)

df = pd.DataFrame(results)
df['FAE_ACC'] = df['FAE_ACC_Mean'].apply(lambda x: f"{x:.4f}") + " ± " + df['FAE_ACC_Std'].apply(lambda x: f"{x:.4f}")
df['FAE_ARI'] = df['FAE_ARI_Mean'].apply(lambda x: f"{x:.4f}") + " ± " + df['FAE_ARI_Std'].apply(lambda x: f"{x:.4f}")
df['FPCA_ACC'] = df['FPCA_ACC_Mean'].apply(lambda x: f"{x:.4f}") + " ± " + df['FPCA_ACC_Std'].apply(lambda x: f"{x:.4f}")
df['FPCA_ARI'] = df['FPCA_ARI_Mean'].apply(lambda x: f"{x:.4f}") + " ± " + df['FPCA_ARI_Std'].apply(lambda x: f"{x:.4f}")
df['Winner_ACC'] = df.apply(lambda row: 'FAE' if row['FAE_ACC_Mean'] > row['FPCA_ACC_Mean'] else 'FPCA', axis=1)
df['Winner_ARI'] = df.apply(lambda row: 'FAE' if row['FAE_ARI_Mean'] > row['FPCA_ARI_Mean'] else 'FPCA', axis=1)

# Display the formatted table
result_table = df[['Dataset', 'N_Samples', 'N_Classes', 'FAE_ACC', 'FPCA_ACC', 'Winner_ACC', 'FAE_ARI', 'FPCA_ARI', 'Winner_ARI']]
print(result_table.to_string(index=False))
print("=" * 120)

# Additional statistics
fae_acc_wins = (df['Winner_ACC'] == 'FAE').sum()
fpca_acc_wins = (df['Winner_ACC'] == 'FPCA').sum()
fae_ari_wins = (df['Winner_ARI'] == 'FAE').sum()
fpca_ari_wins = (df['Winner_ARI'] == 'FPCA').sum()

print(f"\nSummary:")
print(f"  ACC: FAE wins in {fae_acc_wins} datasets, FPCA wins in {fpca_acc_wins} datasets")
print(f"  ARI: FAE wins in {fae_ari_wins} datasets, FPCA wins in {fpca_ari_wins} datasets")
print("=" * 120)


Comparing FAE and FPCA Clustering Performance on UCR Datasets

Dataset: BME
Shape: (180, 128)
Classes: 3

--- Model Training Run 1/5 ---
Training FAE...

--- Training FAE ---
Epoch 10, Loss: 15.3288
Epoch 20, Loss: 7.0923
Epoch 30, Loss: 7.0088
Epoch 40, Loss: 6.9179
Epoch 50, Loss: 6.7722
Epoch 60, Loss: 6.5297
Epoch 70, Loss: 6.3861
Epoch 80, Loss: 6.1109
Epoch 90, Loss: 5.4392
Epoch 100, Loss: 5.1540
Epoch 110, Loss: 5.0559
Epoch 120, Loss: 4.9707
Epoch 130, Loss: 4.8918
Epoch 140, Loss: 4.8174
Epoch 150, Loss: 4.7688
Epoch 160, Loss: 4.7083
Epoch 170, Loss: 4.6164
Epoch 180, Loss: 4.5571
Epoch 190, Loss: 4.5429
Epoch 200, Loss: 4.4926
Epoch 210, Loss: 4.4461
Epoch 220, Loss: 4.4216
Epoch 230, Loss: 4.4074
Epoch 240, Loss: 4.3931
Epoch 250, Loss: 4.3890
Epoch 260, Loss: 4.3417
Epoch 270, Loss: 4.3933
Epoch 280, Loss: 4.3099
Epoch 290, Loss: 4.3485
Epoch 300, Loss: 4.2836
Epoch 310, Loss: 4.3081
Epoch 320, Loss: 4.3041
Epoch 330, Loss: 4.2483
Epoch 340, Loss: 4.2167
Epoch 350, Loss: 

d:\Software\Anaconda\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
d:\Software\Anaconda\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
d:\Software\Anaconda\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
d:\Software\Anaconda\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when

FAE - ACC: 0.5667, ARI: 0.1260
Training FPCA...


d:\Software\Anaconda\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
d:\Software\Anaconda\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
d:\Software\Anaconda\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
d:\Software\Anaconda\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when

FPCA - ACC: 0.4556, ARI: 0.1229

--- Model Training Run 2/5 ---
Training FAE...

--- Training FAE ---
Epoch 10, Loss: 15.8355
Epoch 20, Loss: 7.2119
Epoch 30, Loss: 7.1158
Epoch 40, Loss: 7.0777
Epoch 50, Loss: 7.0405
Epoch 60, Loss: 7.0372
Epoch 70, Loss: 7.0115
Epoch 80, Loss: 6.9775
Epoch 90, Loss: 6.9365
Epoch 100, Loss: 6.9193
Epoch 110, Loss: 6.8746
Epoch 120, Loss: 6.7956
Epoch 130, Loss: 5.7175
Epoch 140, Loss: 5.0554
Epoch 150, Loss: 4.8538
Epoch 160, Loss: 4.7437
Epoch 170, Loss: 4.6503
Epoch 180, Loss: 4.5868
Epoch 190, Loss: 4.5458
Epoch 200, Loss: 4.5185
Epoch 210, Loss: 4.4213
Epoch 220, Loss: 4.3252
Epoch 230, Loss: 4.2135
Epoch 240, Loss: 4.1573
Epoch 250, Loss: 4.0537
Epoch 260, Loss: 4.0225
Epoch 270, Loss: 3.9386
Epoch 280, Loss: 3.9330
Epoch 290, Loss: 3.8712
Epoch 300, Loss: 3.8222
Epoch 310, Loss: 3.7931
Epoch 320, Loss: 3.5547
Epoch 330, Loss: 3.4387
Epoch 340, Loss: 3.3780
Epoch 350, Loss: 3.2998
Epoch 360, Loss: 3.2311
Epoch 370, Loss: 3.1573
Epoch 380, Loss: 3

...

FPCA - ACC: 0.4556, ARI: 0.1265

--- Model Training Run 3/5 ---
Training FAE...

--- Training FAE ---
Epoch 10, Loss: 19.4738
Epoch 20, Loss: 7.4567
Epoch 30, Loss: 7.1391
Epoch 40, Loss: 7.0652
Epoch 50, Loss: 7.0074
Epoch 60, Loss: 6.9651
Epoch 70, Loss: 6.9160
Epoch 80, Loss: 6.7949
Epoch 90, Loss: 6.6295
Epoch 100, Loss: 6.4981
Epoch 110, Loss: 6.4884
Epoch 120, Loss: 6.4274
Epoch 130, Loss: 6.4093
Epoch 140, Loss: 6.3598
Epoch 150, Loss: 6.3223
Epoch 160, Loss: 6.3123
Epoch 170, Loss: 6.2456
Epoch 180, Loss: 6.3290
Epoch 190, Loss: 6.1277
Epoch 200, Loss: 6.0425
Epoch 210, Loss: 6.0198
Epoch 220, Loss: 5.9356
Epoch 230, Loss: 5.5960
Epoch 240, Loss: 4.5787
Epoch 250, Loss: 4.3242
Epoch 260, Loss: 4.1172
Epoch 270, Loss: 3.9980
Epoch 280, Loss: 3.8971
Epoch 290, Loss: 3.8431
Epoch 300, Loss: 3.8152
Epoch 310, Loss: 3.7727
Epoch 320, Loss: 3.7027
Epoch 330, Loss: 3.6752
Epoch 340, Loss: 3.6392
Epoch 350, Loss: 3.6423
Epoch 360, Loss: 3.5391
Epoch 370, Loss: 3.3991
Epoch 380, Loss: 3

d:\Software\Anaconda\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
d:\Software\Anaconda\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
d:\Software\Anaconda\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
d:\Software\Anaconda\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when

FAE - ACC: 0.4833, ARI: 0.1569
Training FPCA...


d:\Software\Anaconda\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
d:\Software\Anaconda\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
d:\Software\Anaconda\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
d:\Software\Anaconda\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when

FPCA - ACC: 0.4556, ARI: 0.1229

--- Model Training Run 4/5 ---
Training FAE...

--- Training FAE ---
Epoch 10, Loss: 19.3375
Epoch 20, Loss: 7.0846
Epoch 30, Loss: 7.0062
Epoch 40, Loss: 6.9261
Epoch 50, Loss: 6.8382
Epoch 60, Loss: 6.7117
Epoch 70, Loss: 6.5398
Epoch 80, Loss: 6.4230
Epoch 90, Loss: 6.2836
Epoch 100, Loss: 5.6850
Epoch 110, Loss: 5.3473
Epoch 120, Loss: 5.1924
Epoch 130, Loss: 5.0961
Epoch 140, Loss: 5.0328
Epoch 150, Loss: 4.8846
